In [20]:
import torch

## 框架真正厉害的点？
可以帮我们把反向传播全部计算好

In [21]:
# Method 1 - recommended
x = torch.randn(3,4,requires_grad=True)
x

tensor([[ 1.0264, -0.3336,  1.2340,  0.7088],
        [-0.6084,  0.9388, -0.4922,  0.0050],
        [-0.0308, -0.3945, -1.1966,  0.0087]], requires_grad=True)

In [22]:
# Method 2
x = torch.randn(3,4)
x.requires_grad = True
x

tensor([[ 0.3705, -0.2516,  1.0966, -0.6605],
        [ 0.9128,  0.8298,  1.6777,  0.6767],
        [ 0.2689,  0.1837,  0.6901,  0.9913]], requires_grad=True)

In [23]:
b = torch.randn(3,4,requires_grad=True)
b

tensor([[-1.6648, -0.7245, -0.5792,  0.6969],
        [-0.9014,  0.7183, -1.4697,  1.0863],
        [ 0.8653,  0.1859,  0.4625,  0.0201]], requires_grad=True)

In [24]:
t = x+b
t

tensor([[-1.2943, -0.9761,  0.5174,  0.0364],
        [ 0.0114,  1.5481,  0.2080,  1.7631],
        [ 1.1342,  0.3695,  1.1526,  1.0114]], grad_fn=<AddBackward0>)

In [25]:
y = t.sum()
y

tensor(5.4817, grad_fn=<SumBackward0>)

In [26]:
y.backward()
y

tensor(5.4817, grad_fn=<SumBackward0>)

In [27]:
b.grad
## 这儿的变量就是b，一元一次函数求导

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

虽然没有指定t的required_grad但是需要用到它，也会默认的

In [28]:
x.requires_grad,b.requires_grad,t.requires_grad

(True, True, True)

In [29]:
# calculate flow - example 
x = torch.rand(1)
y = torch.rand(1)
w = torch.rand(1,requires_grad=True)
y = w*x
z = y+b

In [30]:
# check autograde
x.requires_grad,b.requires_grad,w.requires_grad

(False, True, True)

In [40]:
## again
x = torch.rand(1)
b = torch.rand(1,requires_grad=True)
w = torch.rand(1,requires_grad=True)
y = w*x
z = y+b

In [36]:
# check autograde
x.requires_grad,b.requires_grad,w.requires_grad,y.requires_grad # y is needed

(False, True, True, True)

In [32]:
x.is_leaf,w.is_leaf,b.is_leaf,y.is_leaf,z.is_leaf
# z = wx+b,是否为叶子结点

(True, True, True, False, False)

In [41]:
# back propagation
z.backward(retain_graph=True) # 注意，如果不清空梯度会累加起来


In [42]:
w.grad

tensor([0.6417])

In [43]:
b.grad

tensor([1.])

线性回归试水

In [58]:
# 构造一组输入数据X与对应的标签Y
import numpy as np
x_values = [i for i in range(11)]
x_train  = np.array(x_values,dtype = np.float64)
x_train = x_train.reshape(-1,1)
x_train.shape

(11, 1)

In [59]:
import numpy as np
y_values = [2*i+1 for i in range(11)]
y_train = np.array(y_values,dtype=np.float64)
y_train = y_train.reshape(-1,1)
y_train.shape

(11, 1)

In [50]:
import torch 
import torch.nn as nn

线性回归模型

In [52]:
# 线性回归就是不添加激活函数的全连接层
class LinearRegressionModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(LinearRegressionModel,self).__init__()
        self.linear = nn.Linear(input_dim,output_dim)
        
    def forward(self,x):
        out = self.linear(x)
        return out

In [54]:
input_dim = 1
output_dim = 1

model = LinearRegressionModel(input_dim,output_dim,)

In [55]:
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [56]:
# 制定好参数和损失函数
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
criterion = nn.MSELoss()
# 一般分类任务是交叉熵，回归任务mse

In [61]:
# 训练模型
for epoch in range(epochs):
    epoch+=1
    # 注意转为tensor
    inputs = torch.tensor(x_train)
    labels = torch.tensor(y_train)
    
    # 梯度要清零每一次迭代
    optimizer.zero_grad()
    
    # 前向传播
    outputs = model(inputs)
    
    # 计算损失
    loss = criterion(outputs,labels)
    
    # 返回传播
    loss.backward()
    
    # 更新权重
    optimizer.step()
    
    if epoch % 50 == 0:
        print("epoch {},loss {}".format(epoch,loss.item()))
    
    
    

epoch 50,loss 1.6038194416978513e-06
epoch 100,loss 9.14937288598594e-07
epoch 150,loss 5.220479692980007e-07
epoch 200,loss 2.977829467454285e-07
epoch 250,loss 1.698910097047701e-07
epoch 300,loss 9.69701261510636e-08
epoch 350,loss 5.5252801445249133e-08
epoch 400,loss 3.152409888684815e-08
epoch 450,loss 1.7955974485062143e-08
epoch 500,loss 1.0293295105157085e-08
epoch 550,loss 5.8536993030600115e-09
epoch 600,loss 3.361364520060306e-09
epoch 650,loss 1.920833492263796e-09
epoch 700,loss 1.0967059482780428e-09
epoch 750,loss 6.302759425480531e-10
epoch 800,loss 3.575153784129981e-10
epoch 850,loss 2.1995431864763049e-10
epoch 900,loss 1.3579505575567197e-10
epoch 950,loss 7.115891903497484e-11
epoch 1000,loss 3.915478011262685e-11


LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)